# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
import pandas as pd
import boto3
import sagemaker

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [3]:
# should be the name of directory you created to save your features data
data_dir = 'plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'plagiarism-detector-model'

# upload all data to S3
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print('upload all data to S3 path: ', input_data)

upload all data to S3 path:  s3://sagemaker-eu-west-2-113261559758/plagiarism-detector-model


### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

plagiarism-detector-model/test.csv
plagiarism-detector-model/train.csv
Test passed!


---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [5]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd

import sklearn.externals
import joblib

## TODO: Import any additional libraries you need to define a model
from sklearn.svm import SVC;

# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")
    
    return model


## TODO: Complete the main code
if __name__ == '__main__':
    
    # All of the model parameters and training parameters are sent as arguments
    # when this script is executed, during a training job
    
    # Here we set up an argument parser to easily access the parameters
    parser = argparse.ArgumentParser()

    # SageMaker parameters, like the directories for training data and saving models; set automatically
    #

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [6]:
# your import and estimator code, here
from sagemaker.sklearn.estimator import SKLearn;

output_path = 's3://{}/{}'.format(bucket, prefix);

estimator = SKLearn(entry_point="train.py",
                    source_dir="source_sklearn",
                    sagemaker_session = sagemaker_session,
                    framework_version="0.23-1",
                    #framework_version = '0.20.0',
                    role=role,
                    train_instance_count=1,
                    output_path=output_path,
                    train_instance_type='ml.c4.xlarge');

# Reference: https://stackoverflow.com/questions/61893719/importerror-cannot-import-name-joblib-from-sklearn-externals
# Reference: https://github.com/joblib/joblib/issues/1059

## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [7]:
%%time

# Train your estimator on S3 training data
estimator.fit({'train': input_data});


's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-10-23 16:16:06 Starting - Starting the training job...
2020-10-23 16:16:10 Starting - Launching requested ML instances......
2020-10-23 16:17:16 Starting - Preparing the instances for training...
2020-10-23 16:18:07 Downloading - Downloading input data......
2020-10-23 16:18:38 Training - Downloading the training image..2020-10-23 16:19:16,749 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
2020-10-23 16:19:16,750 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-23 16:19:16,760 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-10-23 16:19:17,179 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-23 16:19:17,810 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-23 16:19:17,822 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-10-23 16:19:17,831 sage

## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [8]:
%%time

# uncomment, if needed
# from sagemaker.pytorch import PyTorchModel


# deploy your model to create a predictor
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.t2.medium');

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


---------------------!CPU times: user 354 ms, sys: 11.3 ms, total: 365 ms
Wall time: 10min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [10]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x);


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [11]:
# Second: calculate the test accuracy

from sklearn.metrics import accuracy_score,f1_score, average_precision_score, confusion_matrix;

accuracy = accuracy = accuracy_score(test_y, test_y_preds);

print(f"Accuracy is {accuracy_score(test_y, test_y_preds):.2f}");

print(f"f1_score is {f1_score(test_y, test_y_preds, zero_division=1)}");

average_precision = average_precision_score(test_y, test_y_preds);

print('Average precision-recall score is: {0:0.2f}'.format(average_precision));

# Reference: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
tn, fp, fn, tp = confusion_matrix(test_y, test_y_preds).ravel()
print('Confusion Matrix - False Positives: ', fp);
print('Confusion Matrix - False Negatives: ', fn);
print('Confusion Matrix - True Positives: ', tp);
print('Confusion Matrix - True Negatives: ', tn);


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

Accuracy is 1.00
f1_score is 1.0
Average precision-recall score is: 1.00
Confusion Matrix - False Positives:  0
Confusion Matrix - False Negatives:  0
Confusion Matrix - True Positives:  15
Confusion Matrix - True Negatives:  10

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

**Answer**:

In a first analysis we verify that the dataset is small, and with few features. The applied model is Support Vector Classification (SVC) model, where the accuracy of the model is 1.00 - (100%), which means that the probability of having some data misclassified is quite low, which means that there aren't **false positives** or **false negatives**.

### Question 2: How did you decide on the type of model to use? 

**Answer**:

Looking at the problem in question I knew previously that it was a classification problem, where the output will be 1 or 0. For this type of problems there are several models in the sklearn library, so we clearly have the possibility to choose the most suitable model for our problem. 
So knowing the data model, I chose as the final model a **Support Vector Classification** (SVC) model, because after some tests with other models (_KNeighborsClassifier_, _MLPClassifier_, among others) I realized that the SVC model, for my data set, was the one that presented the best precision. However, I also ended up using some hyperparameters.
In conclusion, the accuracy of the model is very good.

In [12]:
assessment_reults_df = pd.concat([pd.DataFrame(test_x), pd.DataFrame(test_y), pd.DataFrame(test_y_preds)], axis=1);
assessment_reults_df.columns=['c_1', 'c_2', 'c_3', 'lcs_word', 'class', 'class_predicted'];
print('Result of the prediction :');
print('');
print(assessment_reults_df);
final_assessment_reults_df = assessment_reults_df.loc[assessment_reults_df['class'] != assessment_reults_df['class_predicted']];
print('');
print('');
print('');
print('Prediction Diff:');
print('');
print(final_assessment_reults_df);

Result of the prediction :

         c_1       c_2       c_3  lcs_word  class  class_predicted
0   1.000000  0.984694  0.964103  0.820755      1                1
1   0.765306  0.709898  0.664384  0.621711      1                1
2   0.884444  0.526786  0.340807  0.597458      1                1
3   0.619048  0.250000  0.112299  0.427835      1                1
4   0.920000  0.675676  0.561644  0.775000      1                1
5   0.992674  0.985294  0.981550  0.993056      1                1
6   0.412698  0.112903  0.032787  0.346667      0                0
7   0.462687  0.070000  0.010050  0.189320      0                0
8   0.581152  0.142105  0.037037  0.247423      0                0
9   0.584211  0.142857  0.010638  0.294416      0                0
10  0.566372  0.151786  0.036036  0.258333      0                0
11  0.481481  0.171642  0.067669  0.278912      0                0
12  0.619792  0.251309  0.100000  0.341584      1                1
13  0.921739  0.842795  0.776316  

----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [13]:
# uncomment and fill in the line below!
# <name_of_deployed_predictor>.delete_endpoint()
predictor.delete_endpoint();

### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [14]:
# deleting bucket, uncomment lines below

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '4F4DD16C1CFE76CB',
   'HostId': 'rSbxv3p2gYCxcedKqHTdZDBbGGLssgLGjD9N8QeFXO9rHo3OFJc1k1walXJc3HL11PIWBqUqs2M=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'rSbxv3p2gYCxcedKqHTdZDBbGGLssgLGjD9N8QeFXO9rHo3OFJc1k1walXJc3HL11PIWBqUqs2M=',
    'x-amz-request-id': '4F4DD16C1CFE76CB',
    'date': 'Fri, 23 Oct 2020 16:30:22 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'plagiarism-detector-model/test.csv'},
   {'Key': 'sagemaker-scikit-learn-2020-10-23-16-16-06-680/source/sourcedir.tar.gz'},
   {'Key': 'plagiarism-detector-model/train.csv'},
   {'Key': 'plagiarism-detector-model/sagemaker-scikit-learn-2020-10-23-16-16-06-680/output/model.tar.gz'},
   {'Key': 'plagiarism-detector-model/sagemaker-scikit-learn-2020-10-23-16-16-06-680/debug-output/training_job_end.ts'}]}]

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!

In [26]:
# ZIP
!zip -r project.zip .

  adding: 3_Training_a_Model.ipynb (deflated 74%)
  adding: data.zip (stored 0%)
  adding: plagiarism_data/ (stored 0%)
  adding: plagiarism_data/train.csv (deflated 60%)
  adding: plagiarism_data/test.csv (deflated 59%)
  adding: plagiarism_data/.ipynb_checkpoints/ (stored 0%)
  adding: __pycache__/ (stored 0%)
  adding: __pycache__/helpers.cpython-36.pyc (deflated 42%)
  adding: __pycache__/problem_unittests.cpython-36.pyc (deflated 48%)
  adding: helpers.py (deflated 64%)
  adding: 2_Plagiarism_Feature_Engineering.ipynb (deflated 48%)
  adding: data/ (stored 0%)
  adding: data/g4pD_taskc.txt (deflated 55%)
  adding: data/g2pE_taskc.txt (deflated 51%)
  adding: data/test_info.csv (deflated 78%)
  adding: data/orig_taskc.txt (deflated 51%)
  adding: data/g4pC_taska.txt (deflated 50%)
  adding: data/g0pB_taskb.txt (deflated 52%)
  adding: data/g0pC_taskd.txt (deflated 51%)
  adding: data/file_information.csv (deflated 83%)
  adding: data/g1pA_taske.txt (deflated 50%)
  adding: data/g2p